In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from analysis import PatternDetector



In [2]:
mongo_db = MongoDatabase()
mongo_query_ops = MongoQueryOperations(mongo_db)
sql_db = SQLDatabase()
sql_query_ops = SQLQueryOperations(sql_db)


In [3]:
query = mongo_query_ops.get_recent_transactions(network='Ethereum', limit=1000000)


In [4]:
pattern_detector = PatternDetector()
for transaction in query:
    pattern_detector.process_transaction(transaction)



In [5]:
common_patterns = pattern_detector.get_patterns(min_frequency=100)


In [6]:
common_patterns


[{'sequence': [('0xdac17f958d2ee523a2206206994597c13d831ec7', 'Transfer')],
  'frequency': 111047,
  'sample_tx': ['0x7a8d969efe9a4d5df478764a8ab46350f366c718dbd9f2010d45de7c85ef133a',
   '0xc5aed3fc02f7d438270c2d0c7b1cf186f06b80cf770e686aa5a4cd7130ceef03',
   '0xd7660c6254fdec7a566319bbc61fbe753f77f13adfca3a4acebb73e4d8c905e2',
   '0x364120dd2e354fddba746b39a747e01a2ea632b4a043347922b62cc8305006f5',
   '0x8eae054f628b16ff570eeabb2f27ec933d6403a5ea316506edb5dbefcebebf8e']},
 {'sequence': [('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'Transfer')],
  'frequency': 6911,
  'sample_tx': ['0x9a9a7f8ef7aceb7df6b355cfbfc34f8654dfbe901594d45e4a2d331d950333e9',
   '0xec43606bf6ce2a5c060da0be469a1998057b7c8d2374922db9098d9e7c34308a',
   '0xb9430f4f9e0eb6357f026fecc40934d6109761732a8556afc21619207fb4d442',
   '0xe4ecc56b79020665c7154ebfc421105c39bf8223328ed3b7aed75a9b0976e06c',
   '0xa97ab5cd7dca229e40dfd5e5cf7eaf97bf8c1f492e72a5620dd8819bbeafe49b']},
 {'sequence': [('0xdac17f958d2ee523a22062069

In [7]:
from chains import EthereumProcessor, EthereumQuerier
from database import SQLInsertOperations


In [8]:
querier = EthereumQuerier()
processor = EthereumProcessor(sql_db, mongo_db, querier)


In [9]:
import json
from web3 import Web3
sql_insert_ops = SQLInsertOperations(sql_db)

In [10]:
for pattern in common_patterns:
    for event in pattern['sequence']:
        if event[1] == 'Swap':
            checksum_address = Web3.to_checksum_address(event[0])
            results = sql_query_ops.query_evm_contract_abi('Ethereum', checksum_address)
            try:
                abi = results.get('abi')
            except:
                print('ok')
                
            if abi == None:
                checksum_address = Web3.to_checksum_address(event[0])
                abi = querier.get_contract_abi(checksum_address)
                abi = json.dumps(abi)
                sql_insert_ops.insert_evm_contract_abi('Ethereum', checksum_address, abi)
            
            contract = await processor._process_contract(checksum_address, json.loads(abi))
            


In [11]:
address = Web3.to_checksum_address('0xEBFC31ea16D1080da050E23498cd9503C9b2098A')

In [14]:
address

'0xEBFC31ea16D1080da050E23498cd9503C9b2098A'

In [12]:
result = sql_query_ops.query_evm_contract_abi('Ethereum',address)


In [13]:
result

RealDictRow([('network', 'Ethereum'),
             ('contract_address',
              '0xEBFC31ea16D1080da050E23498cd9503C9b2098A'),
             ('abi',
              '[{"inputs": [], "payable": false, "stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "spender", "type": "address"}, {"indexed": false, "internalType": "uint256", "name": "value", "type": "uint256"}], "name": "Approval", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "sender", "type": "address"}, {"indexed": false, "internalType": "uint256", "name": "amount0", "type": "uint256"}, {"indexed": false, "internalType": "uint256", "name": "amount1", "type": "uint256"}, {"indexed": true, "internalType": "address", "name": "to", "type": "address"}], "name": "Burn", "type": "event"}, {"anonymous": false, 

In [23]:
result.get('abi') == None


False

In [38]:
results = sql_query_ops.query_evm_swap('Ethereum','0xe3cc21af229934034b320502fa55cfa1ab7b1493')

In [54]:
abi = querier.get_contract_abi('0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b')

In [55]:
abi

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickLower',
    'type': 'int24'},
   {'indexed': True,
    'internalType': 'int24',
    'name': 'tickUpper',
    'type': 'int24'},
   {'indexed': False,
    'internalType': 'uint128',
    'name': 'amount',
    'type': 'uint128'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount0',
    'type': 'uint256'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount1',
    'type': 'uint256'}],
  'name': 'Burn',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'owner',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'address',
    'name': 'recipient',
    'type': 'address'},
   {'indexed': True,
    'internalT